In [ ]:
!pip install tensorflow opencv-python kagglehub torch torchvision transformers pillow requests


In [ ]:
import kagglehub

# Download FER2013 dataset once
path = kagglehub.dataset_download("msambare/fer2013")

print("✅ FER2013 dataset path:", path)


Using Colab cache for faster access to the 'fer2013' dataset.
✅ FER2013 dataset path: /kaggle/input/fer2013


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1  # 10% for validation
)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)


Found 25841 images belonging to 7 classes.
Found 2868 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

def build_emotion_model(input_shape=(48, 48, 1), num_classes=7):
    model = Sequential([
        Conv2D(64, (3,3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Dropout(0.25),

        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Dropout(0.25),

        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Dropout(0.3),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

emotion_model = build_emotion_model()
emotion_model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     4,719,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         3,59

 Total params: 5,870,535 (22.39 MB)

 Trainable params: 5,868,743 (22.39 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

EPOCHS = 50

early_stop = EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)
checkpoint = ModelCheckpoint('best_emotion_model.h5', monitor='val_accuracy', save_best_only=True)

history = emotion_model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.2273 - loss: 2.7197

404/404 ━━━━━━━━━━━━━━━━━━━━ 83s 178ms/step - accuracy: 0.2273 - loss: 2.7180 - val_accuracy: 0.2471 - val_loss: 1.8400 - learning_rate: 5.0000e-04
Epoch 2/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.2812 - loss: 1.7594

404/404 ━━━━━━━━━━━━━━━━━━━━ 52s 128ms/step - accuracy: 0.2812 - loss: 1.7593 - val_accuracy: 0.2891 - val_loss: 1.7939 - learning_rate: 5.0000e-04
Epoch 3/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.3213 - loss: 1.6691

404/404 ━━━━━━━━━━━━━━━━━━━━ 52s 129ms/step - accuracy: 0.3213 - loss: 1.6690 - val_accuracy: 0.4248 - val_loss: 1.4943 - learning_rate: 5.0000e-04
Epoch 4/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.3752 - loss: 1.5724

...

404/404 ━━━━━━━━━━━━━━━━━━━━ 51s 126ms/step - accuracy: 0.6548 - loss: 0.9349 - val_accuracy: 0.6429 - val_loss: 0.9813 - learning_rate: 2.5000e-04
Epoch 37/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 52s 129ms/step - accuracy: 0.6502 - loss: 0.9407 - val_accuracy: 0.6350 - val_loss: 0.9841 - learning_rate: 2.5000e-04
Epoch 38/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 50s 125ms/step - accuracy: 0.6533 - loss: 0.9355 - val_accuracy: 0.6406 - val_loss: 0.9692 - learning_rate: 2.5000e-04
Epoch 39/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.6637 - loss: 0.9187

404/404 ━━━━━━━━━━━━━━━━━━━━ 52s 128ms/step - accuracy: 0.6637 - loss: 0.9187 - val_accuracy: 0.6434 - val_loss: 0.9633 - learning_rate: 2.5000e-04
Epoch 40/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 52s 128ms/step - accuracy: 0.6627 - loss: 0.9146 - val_accuracy: 0.6362 - val_loss: 0.9962 - learning_rate: 2.5000e-04
Epoch 41/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 51s 126ms/step - accuracy: 0.6611 - loss: 0.9062 - val_accuracy: 0.6421 - val_loss: 0.9921 - learning_rate: 2.5000e-04
Epoch 42/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 51s 127ms/step - accuracy: 0.6593 - loss: 0.9126 - val_accuracy: 0.6404 - val_loss: 0.9962 - learning_rate: 2.5000e-04
Epoch 43/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.6687 - loss: 0.8929

404/404 ━━━━━━━━━━━━━━━━━━━━ 54s 133ms/step - accuracy: 0.6687 - loss: 0.8929 - val_accuracy: 0.6478 - val_loss: 0.9843 - learning_rate: 2.5000e-04
Epoch 44/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.6760 - loss: 0.8775

404/404 ━━━━━━━━━━━━━━━━━━━━ 54s 134ms/step - accuracy: 0.6760 - loss: 0.8775 - val_accuracy: 0.6500 - val_loss: 0.9642 - learning_rate: 1.2500e-04
Epoch 45/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.6786 - loss: 0.8753

404/404 ━━━━━━━━━━━━━━━━━━━━ 53s 131ms/step - accuracy: 0.6786 - loss: 0.8753 - val_accuracy: 0.6592 - val_loss: 0.9590 - learning_rate: 1.2500e-04
Epoch 46/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 51s 126ms/step - accuracy: 0.6831 - loss: 0.8542 - val_accuracy: 0.6507 - val_loss: 0.9863 - learning_rate: 1.2500e-04
Epoch 47/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 51s 125ms/step - accuracy: 0.6883 - loss: 0.8493 - val_accuracy: 0.6581 - val_loss: 0.9686 - learning_rate: 1.2500e-04
Epoch 48/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 51s 125ms/step - accuracy: 0.6865 - loss: 0.8411 - val_accuracy: 0.6567 - val_loss: 0.9585 - learning_rate: 1.2500e-04
Epoch 49/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 53s 132ms/step - accuracy: 0.6835 - loss: 0.8587 - val_accuracy: 0.6571 - val_loss: 0.9624 - learning_rate: 1.2500e-04
Epoch 50/50
404/404 ━━━━━━━━━━━━━━━━━━━━ 51s 127ms/step - accuracy: 0.6963 - loss: 0.8344 - val_accuracy: 0.6583 - val_loss: 0.9654 - learning_rate: 1.2500e-04


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# Evaluate on test set
loss, acc = emotion_model.evaluate(test_gen)
print(f"Test Accuracy: {acc*100:.2f}%")

# Predict classes
y_true = test_gen.classes
y_pred_prob = emotion_model.predict(test_gen)
y_pred = np.argmax(y_pred_prob, axis=1)
class_labels = list(test_gen.class_indices.keys())

# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Emotion Model")
plt.show()

# Plot Accuracy & Loss
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


 56/113 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.5628 - loss: 1.2158

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model once at the start
emotion_model = load_model("emotion_model.h5")
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Load face cascade once
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_faces(frame, target_size=(48, 48)):
    """
    Detect faces in the frame and preprocess them for the emotion model.
    Returns a list of preprocessed face ROIs.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    rois = [
        cv2.resize(gray[y:y+h, x:x+w], target_size).astype('float32') / 255.0
        for (x, y, w, h) in faces
    ]
    rois = [np.expand_dims(roi, axis=(0, -1)) for roi in rois]  # add batch and channel dims
    return rois

def detect_emotion(frame):
    """
    Predicts the dominant emotion from a frame.
    Returns a dictionary with 'emotion' and 'confidence'.
    """
    rois = preprocess_faces(frame)

    if not rois:  # No faces detected
        return {'emotion': 'neutral', 'confidence': 0.0}

    # Predict emotions for all detected faces
    predictions = [emotion_model.predict(roi, verbose=0) for roi in rois]
    emotions = [(emotion_labels[np.argmax(pred)], float(np.max(pred))) for pred in predictions]

    # Return the face with highest confidence
    best_emotion, best_conf = max(emotions, key=lambda x: x[1])
    return {'emotion': best_emotion, 'confidence': best_conf}


In [ ]:
from PIL import Image
import torch

# scene_labels can be defined once globally to avoid redefining every call
scene_labels = [
    "rain", "clear", "night", "party", "nature", "street",
    "indoor", "outdoor", "beach", "mountain", "office", "home"
]

def predict_image_weather(image_path, model=clip_model, processor=clip_processor):
    """
    Predict climate/context from an image using CLIP.
    Returns the most probable weather/scene label.
    """
    # Open and convert image
    image = Image.open(image_path).convert("RGB")

    # Prepare inputs once
    inputs = processor(text=scene_labels, images=image, return_tensors="pt", padding=True)

    # Forward pass
    with torch.no_grad():  # disable gradients for faster inference
        outputs = model(**inputs)

    probs = outputs.logits_per_image.softmax(dim=1)
    pred_idx = probs.argmax().item()

    return scene_labels[pred_idx]


In [ ]:
import requests

def get_auto_location():
    """
    Detect location automatically (fallback to Coimbatore if unavailable in Colab).
    """
    try:
        res = requests.get("https://ipapi.co/json/").json()
        lat = res.get("latitude")
        lon = res.get("longitude")
        city = res.get("city")
        region = res.get("region")
        country = res.get("country_name")

        # Handle missing data (Colab case)
        if not lat or not lon:
            raise ValueError("No coordinates found")

        print(f"📍 Detected location: {city}, {region}, {country}")
        return {"lat": lat, "lon": lon, "city": city, "state": region, "country": country}
    except Exception as e:
        print(f"⚠️ Location fetch error: {e}")
        # Default fallback
        return {"lat": 11.0168, "lon": 76.9558, "city": "Coimbatore", "state": "Tamil Nadu", "country": "India"}


In [ ]:
import requests

def get_weather(api_key, location=None):
    """
    Fetch current weather using OpenWeatherMap API via lat/lon.
    If location is not provided, automatically detects user location.
    Safe to run in Colab or VS Code.
    Returns a dictionary with city, state, country, climate, and temperature.
    """
    try:
        # Get location automatically if not provided
        if location is None:
            location = get_auto_location()
        lat, lon = location["lat"], location["lon"]

        # Fetch weather data
        url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
        data = requests.get(url, timeout=5).json()  # added timeout for faster failure

        # Check API response
        if data.get("cod") != 200:
            raise Exception(data.get("message", "Weather data not found."))

        # Extract data safely
        weather_main = data.get("weather", [{}])[0].get("main", "clear").lower()
        temp = data.get("main", {}).get("temp", 25)
        city = data.get("name", location.get("city", "Unknown"))

        print(f"🌦 Weather in {city}: {weather_main}, {temp}°C")
        return {
            "city": city,
            "state": location.get("state", "Unknown"),
            "country": location.get("country", "Unknown"),
            "climate": weather_main,
            "temp": temp
        }

    except Exception as e:
        print(f"⚠️ Weather fetch error: {e}")
        # Default fallback
        return {
            "city": location.get("city", "Coimbatore"),
            "state": location.get("state", "Tamil Nadu"),
            "country": location.get("country", "India"),
            "climate": "clear",
            "temp": 25
        }


In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

# Load CLIP once at the start
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define global scene labels once
scene_labels = [
    "indoor", "outdoor", "party", "nature", "street", "night", "office",
    "home", "restaurant", "beach", "concert", "mountain", "park"
]

def classify_scene(image_path, model=clip_model, processor=clip_processor, labels=scene_labels):
    """
    Classifies the scene of an image using CLIP.
    Returns the most probable scene label.
    """
    image = Image.open(image_path).convert("RGB")

    # Prepare inputs
    inputs = processor(text=labels, images=image, return_tensors="pt", padding=True)

    # Forward pass with no grad to speed up inference
    with torch.no_grad():
        outputs = model(**inputs)

    probs = outputs.logits_per_image.softmax(dim=1)
    pred_idx = probs.argmax().item()

    return labels[pred_idx]


In [ ]:
def recommend_song(emotion, scene, user_climate, image_climate):
    """
    Recommends songs based on detected emotion, scene, user climate, and image climate.
    Returns a list of songs and context description.
    """
    # Expanded base songs with more variations
    base_songs = {
        "happy": [
            "Happy – Pharrell Williams", "Good Time – Owl City", "Best Day Of My Life – American Authors",
            "Can't Stop the Feeling – Justin Timberlake", "Walking on Sunshine – Katrina & The Waves"
        ],
        "sad": [
            "Fix You – Coldplay", "Let Her Go – Passenger", "Someone Like You – Adele",
            "Skinny Love – Bon Iver", "Say Something – A Great Big World"
        ],
        "angry": [
            "In The End – Linkin Park", "Stronger – Kanye West", "Believer – Imagine Dragons",
            "Break Stuff – Limp Bizkit", "Killing In The Name – Rage Against The Machine"
        ],
        "fear": [
            "Demons – Imagine Dragons", "Creep – Radiohead", "The Night We Met – Lord Huron",
            "Disturbia – Rihanna", "Bury a Friend – Billie Eilish"
        ],
        "surprise": [
            "Adventure of a Lifetime – Coldplay", "Wake Me Up – Avicii",
            "On Top of the World – Imagine Dragons", "Good Life – OneRepublic"
        ],
        "neutral": [
            "Counting Stars – OneRepublic", "Perfect – Ed Sheeran", "Let It Be – The Beatles",
            "Viva La Vida – Coldplay", "Ho Hey – The Lumineers"
        ],
        "disgust": [
            "Radioactive – Imagine Dragons", "Whatever It Takes – Imagine Dragons",
            "Toxic – Britney Spears", "Bad Guy – Billie Eilish"
        ]
    }

    # Context modifiers for climate and scene
    context_modifiers = {
        "rain": "Acoustic / calm tracks fit rainy moods.",
        "clear": "Bright songs match the sunny vibe.",
        "night": "Chill or lo-fi tracks are perfect for the night.",
        "party": "Upbeat pop or EDM songs suit party scenes.",
        "nature": "Soft indie or instrumental fits nature views.",
        "indoor": "Relaxed indoor tracks for a cozy atmosphere.",
        "outdoor": "Energetic tracks for outdoor vibes.",
        "beach": "Tropical or reggae tunes for beach moods.",
        "concert": "Live or high-energy songs match concert vibes."
    }

    # Pick songs based on emotion
    songs = base_songs.get(emotion.lower(), base_songs['neutral'])

    # Prioritize image climate over user climate
    mood_context = context_modifiers.get(image_climate.lower(),
                                         context_modifiers.get(user_climate.lower(), ""))

    # Add scene-based song variation (optional: shuffle for more randomness)
    if scene.lower() in context_modifiers:
        mood_context += f" Scene suggests: {context_modifiers[scene.lower()]}"

    # Print summary
    print(f"\n🎭 Emotion: {emotion}")
    print(f"🏙 Scene/Context: {scene}")
    print(f"🌦 User Climate: {user_climate}")
    print(f"☁️ Image Climate: {image_climate}\n")

    print("🎧 Recommended songs:")
    for s in songs:
        print("  •", s)
    if mood_context:
        print("\n💡 Context:", mood_context)

    # Return songs and context for further use
    return songs, mood_context


In [ ]:
# -------------------------------
# 1️⃣ Load Input Image
# -------------------------------
img_path = "test_surprise.jpeg"
frame = cv2.imread(img_path)

# -------------------------------
# 2️⃣ Detect emotion
# -------------------------------
emotion_result = detect_emotion(frame)
emotion = emotion_result['emotion']
print(f"Detected Emotion: {emotion_result}")

# -------------------------------
# 3️⃣ Get user location & weather
# -------------------------------
location = get_auto_location()  # Auto-detect user location
weather_result = get_weather("e37720b4cbebdd5c37df9ae780b8688d
", location)
user_climate = weather_result['climate']
print(f"User Location & Climate: {weather_result}")

# -------------------------------
# 4️⃣ Classify scene/context from image
# -------------------------------
scene_result = classify_scene(img_path)
print(f"Scene/Context Detected: {scene_result}")

# -------------------------------
# 5️⃣ Predict weather/climate from image
# -------------------------------
image_climate = predict_image_weather(img_path)
print(f"Image Climate Detected: {image_climate}")

# -------------------------------
# 6️⃣ Recommend songs
# -------------------------------
songs, mood_context = recommend_song(
    emotion=emotion,
    scene=scene_result,
    user_climate=user_climate,
    image_climate=image_climate
)
